In [0]:
import pyspark
from pyspark.sql import SparkSession as ss
from pyspark.sql.functions import *



In [0]:
spark=ss.builder.appName('Walmart').getOrCreate()

In [0]:
walmart_df=spark.read.csv('dbfs:/FileStore/shared_uploads/ananthbulusu@gmail.com/Walmart/Walmart_Stock_1972_2022.csv',header=True,inferSchema=True)

In [0]:
walmart_df.select(min('date'),max('date')).show()

+-------------------+-------------------+
|          min(date)|          max(date)|
+-------------------+-------------------+
|1972-08-25 00:00:00|2022-06-24 00:00:00|
+-------------------+-------------------+



In [0]:
df_cols=walmart_df.columns
df_cols

Out[5]: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

In [0]:
walmart_df.printSchema()
sch=walmart_df.schema
sch

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)

Out[6]: StructType(List(StructField(Date,TimestampType,true),StructField(Open,DoubleType,true),StructField(High,DoubleType,true),StructField(Low,DoubleType,true),StructField(Close,DoubleType,true),StructField(Adj Close,DoubleType,true),StructField(Volume,IntegerType,true)))

In [0]:
walmart_df.show(5,False)

+-------------------+--------+--------+--------+--------+---------+-------+
|Date               |Open    |High    |Low     |Close   |Adj Close|Volume |
+-------------------+--------+--------+--------+--------+---------+-------+
|1972-08-25 00:00:00|0.063477|0.064697|0.063477|0.064453|0.036175 |2508800|
|1972-08-28 00:00:00|0.064453|0.064941|0.064209|0.064209|0.036038 |972800 |
|1972-08-29 00:00:00|0.063965|0.063965|0.063477|0.063477|0.035627 |1945600|
|1972-08-30 00:00:00|0.063477|0.063477|0.062988|0.063477|0.035627 |409600 |
|1972-08-31 00:00:00|0.062988|0.062988|0.0625  |0.0625  |0.035079 |870400 |
+-------------------+--------+--------+--------+--------+---------+-------+
only showing top 5 rows



In [0]:
rows1=walmart_df.collect()[:5]
list(rows1[1][:])

Out[8]: [datetime.datetime(1972, 8, 28, 0, 0),
 0.064453,
 0.064941,
 0.064209,
 0.064209,
 0.036038,
 972800]

In [0]:
walmart_df.describe().show()

+-------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+
|summary|             Open|             High|              Low|             Close|         Adj Close|           Volume|
+-------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+
|  count|            12566|            12566|            12566|             12566|             12566|            12566|
|   mean|36.10611792996976|36.44602781020214|35.77185328473657|36.112548871876605|29.398180219640228|7549621.685500557|
| stddev|37.98339065107896| 38.3087771989581| 37.6805449330805| 37.99106039682545|35.301829831413336| 6235869.46767922|
|    min|         0.015625|         0.015625|         0.014404|          0.015625|          0.008823|                0|
|    max|           160.25|       160.770004|       159.070007|        159.869995|        159.290985|        131833600|
+-------+-----------------+-------------

In [0]:
summary = walmart_df.describe()
summary.select(summary['summary'],
                  format_number(summary['Open'].cast('float'), 2).alias('Open'),
                  format_number(summary['High'].cast('float'), 2).alias('High'),
                  format_number(summary['Low'].cast('float'), 2).alias('Low'),
                  format_number(summary['Close'].cast('float'), 2).alias('Close'),
                  format_number(summary['Volume'].cast('int'),0).alias('Volume')
                 ).show()

+-------+---------+---------+---------+---------+-----------+
|summary|     Open|     High|      Low|    Close|     Volume|
+-------+---------+---------+---------+---------+-----------+
|  count|12,566.00|12,566.00|12,566.00|12,566.00|     12,566|
|   mean|    36.11|    36.45|    35.77|    36.11|  7,549,621|
| stddev|    37.98|    38.31|    37.68|    37.99|  6,235,869|
|    min|     0.02|     0.02|     0.01|     0.02|          0|
|    max|   160.25|   160.77|   159.07|   159.87|131,833,600|
+-------+---------+---------+---------+---------+-----------+



In [0]:
summary = walmart_df.describe()
summary.alias('s').select('s.summary',
                  format_number(col('s.Open').cast('float'), 2).alias('Open'),
                  format_number(summary['High'].cast('float'), 2).alias('High'),
                  format_number(summary['Low'].cast('float'), 2).alias('Low'),
                  format_number(summary['Close'].cast('float'), 2).alias('Close'),
                  format_number(summary['Volume'].cast('int'),0).alias('Volume')
                 ).show()

+-------+---------+---------+---------+---------+-----------+
|summary|     Open|     High|      Low|    Close|     Volume|
+-------+---------+---------+---------+---------+-----------+
|  count|12,566.00|12,566.00|12,566.00|12,566.00|     12,566|
|   mean|    36.11|    36.45|    35.77|    36.11|  7,549,621|
| stddev|    37.98|    38.31|    37.68|    37.99|  6,235,869|
|    min|     0.02|     0.02|     0.01|     0.02|          0|
|    max|   160.25|   160.77|   159.07|   159.87|131,833,600|
+-------+---------+---------+---------+---------+-----------+



In [0]:
hv_ratio_df=walmart_df.withColumn('HV Ratio', walmart_df['High']/walmart_df['Volume']).select(['HV Ratio'])
hv_ratio_df.show()

+--------------------+
|            HV Ratio|
+--------------------+
|2.578802614795918...|
|6.675678453947369E-8|
|3.287674753289473E-8|
|1.549731445312500...|
|7.236672794117647E-8|
|       2.46046875E-7|
|1.118394886363636...|
|       2.46046875E-7|
|5.348845108695652...|
|9.390024038461538E-8|
|6.808159722222221E-8|
|1.021223958333333...|
|4.084895833333333E-7|
|             4.94E-8|
|      1.230234375E-7|
|5.880952380952380...|
|2.008965163934426E-8|
|1.211171874999999...|
|    4.29150390625E-8|
|6.872012867647059E-8|
+--------------------+
only showing top 20 rows



In [0]:
walmart_df.orderBy(col('High').desc()).collect()[0]['Date']

Out[13]: datetime.datetime(2022, 4, 21, 0, 0)

In [0]:
walmart_df.select(mean('Close')).alias('Mean_close').show()

+------------------+
|        avg(Close)|
+------------------+
|36.112548871876605|
+------------------+



In [0]:
#walmart_df.select((sum('Close')/count('Close')).alias('mean')).show()
walmart_df.select((sum(col('Close'))/count(col('Close'))).alias('mean')).show()

+------------------+
|              mean|
+------------------+
|36.112548871876605|
+------------------+



In [0]:
walmart_df.select(max(col('Volume')).alias('max'),min(col('Volume')).alias('min')).show()

+---------+---+
|      max|min|
+---------+---+
|131833600|  0|
+---------+---+



In [0]:
(walmart_df.where(col('High')>80).count()/walmart_df.count() )*100

Out[17]: 10.607989813783226

In [0]:
walmart_df.corr('High','Low')

Out[18]: 0.9998742281130422

In [0]:
walmart_df.select([year('Date').alias('Year'),'High']).groupBy('Year').max('High').orderBy(col('Year').desc()).show()

+----+----------+
|Year| max(High)|
+----+----------+
|2022|160.770004|
|2021|152.570007|
|2020|153.660004|
|2019|125.379997|
|2018|109.980003|
|2017|100.129997|
|2016| 75.190002|
|2015| 90.970001|
|2014| 88.089996|
|2013| 81.370003|
|2012| 77.599998|
|2011|      60.0|
|2010|     56.27|
|2009| 57.509998|
|2008| 63.849998|
|2007| 51.439999|
|2006| 52.150002|
|2005| 54.599998|
|2004| 61.310001|
|2003| 60.200001|
+----+----------+
only showing top 20 rows



In [0]:
walmart_df.select([month('Date').alias('Month'),'Close']).groupBy('Month').avg('Close').orderBy(col('Month').desc()).show()

+-----+------------------+
|Month|        avg(Close)|
+-----+------------------+
|   12| 36.53386848481972|
|   11| 36.44846344062804|
|   10| 35.25492305877033|
|    9| 35.04383120492605|
|    8| 35.36014060733945|
|    7| 35.73718895173742|
|    6| 36.36809093708919|
|    5| 36.72108932071901|
|    4|37.496649986447224|
|    3| 36.50454348401828|
|    2|36.259627149269285|
|    1| 35.66998774855494|
+-----+------------------+

